In [1]:
%pwd

'C:\\Caba2012\\workspace\\Caba_dataprocessing'

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
titanic = pd.read_csv('dataset/titanic3.csv')
titanic.head(2)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"


#### ○ null 처리

* Age : null값 263개를 갖고 있음
* Cabin : null값 1014개를 갖고 있음
* Embarked : null값 2개를 갖고 있음
* fare

In [5]:
titanic.name

0                         Allen, Miss. Elisabeth Walton
1                        Allison, Master. Hudson Trevor
2                          Allison, Miss. Helen Loraine
3                  Allison, Mr. Hudson Joshua Creighton
4       Allison, Mrs. Hudson J C (Bessie Waldo Daniels)
                             ...                       
1304                               Zabour, Miss. Hileni
1305                              Zabour, Miss. Thamine
1306                          Zakarian, Mr. Mapriededer
1307                                Zakarian, Mr. Ortin
1308                                 Zimmerman, Mr. Leo
Name: name, Length: 1309, dtype: object

In [6]:
# 1) Age - Name에서 Title(칭호)을 추출해서 이에 기반한 Age의 평균값을 통해
#    null값을 메워줌
# 1-1) titanic에 'Title' 칼럼 삽입
titanic['Title'] = titanic['name'].apply(lambda x : x.split(',')[1].split('.')[0].strip())
# Mr=성인 남성 / Master=12세 미만의 남성 / Miss=미혼 여성 / Mrs=기혼 여성
for i in range(len(titanic)):
    titanic.iloc[i, 14] = titanic.iloc[i, 14].replace('Dr', 'Mr') 
    titanic.iloc[i, 14] = titanic.iloc[i, 14].replace('Rev', 'Mr') # Rev : 목사 [Mr]
    titanic.iloc[i, 14] = titanic.iloc[i, 14].replace('Col', 'Mr') # Col : 대령[Mr]
    titanic.iloc[i, 14] = titanic.iloc[i, 14].replace('Major', 'Mr') # Major : 소령[Mr]
    titanic.iloc[i, 14] = titanic.iloc[i, 14].replace('Mlle', 'Miss') # Mlle : 마드모아젤[Miss]
    titanic.iloc[i, 14] = titanic.iloc[i, 14].replace('Mme', 'Mrs') # Mme : 마담[Mrs]
    titanic.iloc[i, 14] = titanic.iloc[i, 14].replace('Sir', 'Mr') # Sir : 모르는 남성[Mr]
    titanic.iloc[i, 14] = titanic.iloc[i, 14].replace('Don', 'Mr') # Don : 고위층 남성[Mr]
    titanic.iloc[i, 14] = titanic.iloc[i, 14].replace('Lady', 'Mrs') # Lady : 레이디[Mrs(Name 인터넷 검색)]
    titanic.iloc[i, 14] = titanic.iloc[i, 14].replace('Capt', 'Mr') # Capt : 대위[Mr]
    titanic.iloc[i, 14] = titanic.iloc[i, 14].replace('the Countess', 'Mrs') # the Countess : 카운티스[Mrs(Name 인터넷 검색)]
    titanic.iloc[i, 14] = titanic.iloc[i, 14].replace('Ms', 'Mrs') # Ms : [Mrs (Name 인터넷 검색 -> 1911년 결혼 / 타이타닉 침몰 = 1912년))]
    titanic.iloc[i, 14] = titanic.iloc[i, 14].replace('Jonkheer', 'Mr')
titanic.iloc[796, 14].replace('Mr', 'Mrs')  # 'Dr' 중 female이었던 인물[796 = Mrs(Name 인터넷 검색)]

# 1-2) Age값이 null일 경우(최소값 이상~최대값이하가 아닐 경우) -> TItle을 기반으로 한 평균나이로 대체함
for i in range(len(titanic)):
    if ((titanic.iloc[i, 4] >= titanic.age.min()) & (titanic.iloc[i, 5] <= titanic.age.max())) == False:
        if titanic.iloc[i, 14] == 'Mr':
            titanic.iloc[i, 4] = titanic[titanic.Title == 'Mr'].age.mean()
        elif titanic.iloc[i, 14] == 'Master':
            titanic.iloc[i, 4] = titanic[titanic.Title == 'Master'].age.mean()
        elif titanic.iloc[i, 14] == 'Miss':
            titanic.iloc[i, 4] = titanic[titanic.Title == 'Miss'].age.mean()
        elif titanic.iloc[i, 14] == 'Mrs':
            titanic.iloc[i, 4] = titanic[titanic.Title == 'Mrs'].age.mean()
            
# 2) Embarked - null값은 가장 빈도수가 높은 S(Southhampoton)으로 채움
titanic['embarked'] = titanic['embarked'].fillna('S')

## Q. 타이타닉 생존자 예측모델 개발을 위한 Titanic 분석용 데이터셋

#### Titanic data 전처리
- 분석 데이터 : titanic3.csv
- 재사용 가능한 전처리 사용자 함수 작성 하여 일괄 전처리
Null 값 처리 : Age는 평균나이, 나머지 칼럼은 'N'값으로 변경
Cabin 칼럼은 선실 번호 중 선실 등급을 나타내는 첫 번째 알파벳이 중요하며 앞문자만 추출
문자열 칼럼 레이블 인코딩
불필요한 속성 칼럼 삭제
- 탐색적 분석 필수 항목 : groupby() 또는 sns.barplot() 사용하여 성별 생존자, 객실 등급별 성별 생존자, 연령별(카테고리 분류) 생존자
- 통계적, 시각적 탐색을 통한 다양한 인사이트 도출
- 파생변수 : 분석에 필요한 파생변수 2개 이상 생성
- 탐색 결과에 대한 종합 의견

#### 컬럼 정보

- survived : 생존여부(1: 생존, 0 : 사망)
- pclass : 승선권 클래스(1 : 1st, 2 : 2nd ,3 : 3rd)
- name : 승객 이름
- sex : 승객 성별
- age : 승객 나이
- sibsp : 동반한 형제자매, 배우자 수
- parch : 동반한 부모, 자식 수
- ticket : 티켓의 고유 넘버
- fare 티켓의 요금
- cabin : 객실 번호
- embarked : 승선한 항구명(C : Cherbourg, Q : Queenstown, S : Southampton)
- boat
- body
- home.dest

### 데이터셋 개선을 위한 시사점
* 변수 = 'age_cat','male','female','fare_cat','family',{town_C','town_Q','town_S'}
* age의 Null 값을 평균값으로 대체하면 전체적인 데이터의 왜곡이 심함을 확인
* pclass는 fare_cat이랑 같이 모델에 넣을 경우 정확도가 떨어지고(0.82) 각각 넣었을 때는 fare_cat을 넣었을 때의 정확도가 더 높음(0.82, 3% 차이). 이상치에 가까울 정도로 요금이 높은 사람의 경우 사망률이 3클래스 승객과 비슷한 수치를 보이는 점이 pclass 변수에서는 반영이 되지 않았던 것이 원인으로 추측
* sex, embarked 변수로 집어넣은 것보다 원핫 인코더(dummies)로 처리해서 넣는 것이 정확도를 대략 5% 정도 높여주며 디시전 트리의 분기를 더 쉽게하는 효과 확인
* parch와 sibsp를 각각 변수에 적용하면  의미 있는 양상이 보이지 않고 정확도를 떨어뜨리지만 두 변수를 합쳐서 family라는 파생변수를 생성하면 생존율이 높은 여성 승객일지라도 가족 구성원 수가 많으면 생존율이 낮아지는 것을 확인

### 전처리 내역
- age null 처리방법 변경 : 평균값 대체 > 삭제
- pclass와 fare_cat중 분석변수 선택
- embarked 원핫인코딩

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# 독립변수, 종속변수 분리
y_t_df = titanic_df['survived'] # 종속변수
X_t_df = titanic_df.drop('survived', axis = 1) # 독립변수

# 독립변수 정규화
# X_t_df = preprocessing.StandardScaler().fit(X_t_df).transform(X_t_df)

# 학습용 데이터와 평가용 데이터를 8:2로 분리
X_train, X_test, y_train, y_test = train_test_split(X_t_df, y_t_df, test_size = 0.2,
                                                   random_state = 11)

print(X_train.shape)
print(X_test.shape)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
accuracy_lr = accuracy_score(y_test, lr_pred)
print('Logistic Regression 예측 정확도 :% 1.2f ' % accuracy_lr)